## ETL user_review - Informe de Extracción, Transformación y Carga de Datos:

En esta sección, nos sumergiremos en el proceso de Extracción, Transformación y Carga (ETL) de los datos provenientes del conjunto de reseñas de usuarios australianos de los juegos en la plataforma **Steam**. El objetivo principal es preparar estos datos para su análisis, asegurándonos de que sean aptos y consistentes.

Comenzaremos importando las bibliotecas esenciales. Asegúrese de tener estas bibliotecas instaladas previamente para garantizar una ejecución sin contratiempos.

A lo largo del informe, nos enfocaremos en abordar problemas potenciales en los datos, aplicar técnicas de limpieza y preprocesamiento, y finalmente, almacenar los datos transformados para futuras exploraciones y análisis.


⚠️ **Asegúrese de instalar las siguientes bibliotecas antes de ejecutar el código**

- pandas
- numpy
- gdown
- langdetect
- nltk

Puede instalar estas bibliotecas debe abrir una terminal o ventana de línea de comandos y ejecutar el siguiente comando:

In [ ]:
pip install pandas numpy gdown langdetect nltk

In [ ]:
# Importamos pandas para el análisis de datos tabulares
import pandas as pd

# NumPy proporciona soporte para arreglos y matrices multidimensionales
import numpy as np

# El módulo os permite interactuar con el sistema operativo
import os

# JSON es un formato común para el intercambio de datos, y Python tiene soporte incorporado para trabajar con JSON
import json

# Langdetect es una biblioteca para detectar automáticamente el idioma en el que está escrito un texto
from langdetect import detect

# NLTK (Natural Language Toolkit) es una plataforma para construir programas Python para trabajar con datos de lenguaje humano
import nltk

# SentimentIntensityAnalyzer es una herramienta en NLTK para análisis de sentimientos
from nltk.sentiment import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings("ignore")

import ast

from pandas.io.parquet import to_parquet

from pandas import json_normalize

import utils


⚠️**Nota: Este comando descarga el modelo de análisis de sentimiento de NLTK y solo debe ejecutarse la primera vez que se utiliza.**

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

### 1. Cargar el conjunto de datos original

Fuente de datos: **australian_user_reviews.json**

Es importante señalar que el archivo inicial presentaba inconsistencias en su formato JSON, lo que llevó a su corrupción. Para resolver este problema, utilicé las herramientas "codebeautify" y "notepad++" para reparar el archivo. La versión reparada ahora se encuentra almacenada en Google Drive con acceso compartido. Puede **descargar automáticamente** el archivo ejecutando las siguientes líneas de código (recomendado).

En caso de preferirlo, puede acceder al enlace a continuación para descargar manualmente el archivo:
Datasets: https://bit.ly/3On6yYB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_json_lines(file_path):

    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(ast.literal_eval(line))
    return pd.DataFrame(data)

In [ ]:
#Carga y muestra el archivo
df_reviews = load_json_lines('/content/drive/MyDrive/JSON/australian_user_reviews.json')


### 2. Explorar y entender el conjunto de datos

Exploramos y entendemos la estructura del conjunto de datos, revisando las primeras filas, información general y estadísticas descriptivas.

In [ ]:
# Mostrar las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
df_reviews.head()

Primeras filas del DataFrame:


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


La visualización de las primeras 5 filas del conjunto de datos muestra tres columnas principales: 'user_id', 'user_url', y 'reviews'. Aquí hay algunos comentarios al respecto:

- user_id: Esta columna parece contener identificadores únicos para cada usuario.

- user_url: Contiene enlaces a los perfiles de los usuarios en la plataforma Steam.

- reviews: Esta columna parece contener información detallada sobre las revisiones realizadas por cada usuario. Cada entrada es un diccionario o estructura similar, que incluye detalles como la fecha de publicación, contenido de la revisión, etc.

Definitivamente la columna 'reviews' contiene datos estructurados (formato JSON, puede revelar información adicional sobre las revisiones de los usuarios).

In [ ]:
# Explora columna user_id del DataFrame
df_reviews.info  # Se observa que la columna reviews es una columna anidada

<bound method DataFrame.info of                  user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1                js41637               http://steamcommunity.com/id/js41637   
2              evcentric             http://steamcommunity.com/id/evcentric   
3                  doctr                 http://steamcommunity.com/id/doctr   
4              maplemage             http://steamcommunity.com/id/maplemage   
...                  ...                                                ...   
25794  76561198306599751  http://steamcommunity.com/profiles/76561198306...   
25795           Ghoustik              http://steamcommunity.com/id/Ghoustik   
25796  76561198310819422  http://steamcommunity.com/profiles/76561198310...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

                   

In [ ]:
def eliminar_duplicados(df_reviews):
    # Aplana la columna 'reviews' utilizando json_normalize
    flattened_reviews = json_normalize(df_reviews['reviews'])

    # Agrega 'user_id' al principio de las columnas aplanadas
    flattened_reviews = pd.concat([df_reviews['user_id'], flattened_reviews], axis=1)

    # Elimina duplicados basados en 'user_id'
    user_reviews = flattened_reviews[~flattened_reviews.duplicated(subset='user_id')]

    return user_reviews

# Aplica la función eliminar_duplicados al DataFrame original
user_reviews = eliminar_duplicados(df_reviews)

# Muestra el DataFrame resultante
user_reviews

,user_id,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
25794,76561198306599751,"{'funny': '', 'posted': 'Posted May 31.', 'las...",None,None,None,None,None,None,None,None,None
25795,Ghoustik,"{'funny': '', 'posted': 'Posted June 17.', 'la...",None,None,None,None,None,None,None,None,None
25796,76561198310819422,"{'funny': '1 person found this review funny', ...",None,None,None,None,None,None,None,None,None
25797,76561198312638244,"{'funny': '', 'posted': 'Posted July 21.', 'la...","{'funny': '', 'posted': 'Posted July 10.', 'la...","{'funny': '', 'posted': 'Posted July 10.', 'la...","{'funny': '', 'posted': 'Posted July 8.', 'las...",None,None,None,None,None,None


In [ ]:
# Se utiliza pd.melt para transformar las columnas en filas conservando el 'user_id''
user_reviews = pd.melt(user_reviews, id_vars=['user_id'],
                       value_vars=user_reviews.columns[2:11],
                       value_name='reviews')
user_reviews.head()

,user_id,variable,reviews
0,76561197970982479,1,"{'funny': '', 'posted': 'Posted July 15, 2011...."
1,js41637,1,"{'funny': '', 'posted': 'Posted September 8, 2..."
2,evcentric,1,"{'funny': '', 'posted': 'Posted December 4, 20..."
3,doctr,1,"{'funny': '', 'posted': 'Posted July 28, 2012...."
4,maplemage,1,"{'funny': '1 person found this review funny', ..."


Al hacer esto último se puede ver que quedan registros None. Esto ocurre porque hay usuarios que hicieron mas reviews que otros. En este ejemplo se puede ver este caso:

In [ ]:
user_reviews[user_reviews['user_id']=='76561197970982479']

,user_id,variable,reviews
0,76561197970982479,1,"{'funny': '', 'posted': 'Posted July 15, 2011...."
25485,76561197970982479,2,"{'funny': '', 'posted': 'Posted April 21, 2011..."
50970,76561197970982479,3,None
76455,76561197970982479,4,None
101940,76561197970982479,5,None
127425,76561197970982479,6,None
152910,76561197970982479,7,None
178395,76561197970982479,8,None
203880,76561197970982479,9,None


Se eliminan los registros que tienen None en 'reviews'.

In [ ]:
# Se eliminan las filas con valor None
user_reviews = user_reviews.dropna()
# Se verifica que solo queden el 'user_id' con la cantidad de diccionarios que le corresponde
user_reviews[user_reviews['user_id']=='76561197970982479']

,user_id,variable,reviews
0,76561197970982479,1,"{'funny': '', 'posted': 'Posted July 15, 2011...."
25485,76561197970982479,2,"{'funny': '', 'posted': 'Posted April 21, 2011..."


En este punto ya es posible convertir cada diccionario en columna.

In [ ]:
# Se separan por columnas cada una de las claves de 'reviews'
df_reviews = user_reviews['reviews'].apply(pd.Series, dtype='object')
df_reviews = df_reviews.add_prefix('reviews_')
df_reviews.head()

,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
1,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
2,,"Posted December 4, 2015.","Last edited December 5, 2015.",370360,No ratings yet,True,"""Run for fun? What the hell kind of fun is that?"""
3,,"Posted July 28, 2012.",,20920,1 of 1 people (100%) found this review helpful,True,"Really Really Really Great Game, very good sto..."
4,1 person found this review funny,"Posted December 23, 2013.",,211820,12 of 16 people (75%) found this review helpful,True,"It's like Terraria, you play for 9 hours strai..."


En el procesamiento anterior, se puede ver que la columna de 'user_id' y 'user_url' se perdió nuevamente, por lo que se vuelve a concatenar.

In [ ]:
# Se une con el 'user_id' y 'user_url'
df_reviews = pd.concat([user_reviews[['user_id']], df_reviews], axis=1)
df_reviews.head()

,user_id,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
1,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
2,evcentric,,"Posted December 4, 2015.","Last edited December 5, 2015.",370360,No ratings yet,True,"""Run for fun? What the hell kind of fun is that?"""
3,doctr,,"Posted July 28, 2012.",,20920,1 of 1 people (100%) found this review helpful,True,"Really Really Really Great Game, very good sto..."
4,maplemage,1 person found this review funny,"Posted December 23, 2013.",,211820,12 of 16 people (75%) found this review helpful,True,"It's like Terraria, you play for 9 hours strai..."


Se observa que hay valores faltantes en algunas columnas, pero no estan como nulos, probablemente deben tener un espacio. Se compueba esto.

In [ ]:
df_reviews['reviews_last_edited'][0]

''

Se reemplazar esos espacios como valores nulos.

In [ ]:
df_reviews.replace('', None, inplace=True)
df_reviews.head()

,user_id,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,None,"Posted July 15, 2011.",None,22200,No ratings yet,True,It's unique and worth a playthrough.
1,js41637,None,"Posted September 8, 2013.",None,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
2,evcentric,None,"Posted December 4, 2015.","Last edited December 5, 2015.",370360,No ratings yet,True,"""Run for fun? What the hell kind of fun is that?"""
3,doctr,None,"Posted July 28, 2012.",None,20920,1 of 1 people (100%) found this review helpful,True,"Really Really Really Great Game, very good sto..."
4,maplemage,1 person found this review funny,"Posted December 23, 2013.",None,211820,12 of 16 people (75%) found this review helpful,True,"It's like Terraria, you play for 9 hours strai..."


Se analizan los tipos de datos y los nulos que quedaron luego de desanidar la columna 'reviews'.

In [ ]:
    print(df_reviews.info())

    print("\nValores nulos por columna:")
    print(df_reviews.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32973 entries, 0 to 229095
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              32973 non-null  object
 1   reviews_funny        3864 non-null   object
 2   reviews_posted       32973 non-null  object
 3   reviews_last_edited  4043 non-null   object
 4   reviews_item_id      32973 non-null  object
 5   reviews_helpful      32973 non-null  object
 6   reviews_recommend    32973 non-null  bool  
 7   reviews_review       32955 non-null  object
dtypes: bool(1), object(7)
memory usage: 3.1+ MB
None

Valores nulos por columna:
user_id                    0
reviews_funny          29109
reviews_posted             0
reviews_last_edited    28930
reviews_item_id            0
reviews_helpful            0
reviews_recommend          0
reviews_review            18
dtype: int64


Se observa entre un 86 a 89% de faltantes de datos en las columnas 'reviews_funny' y 'reviews_last_edited' por lo que se decide eliminar estas columnas. Por otra parte hay un 5% de faltantes de datos en la columna propiamente de reviews, pero no se eliminarán esos registros porque se considerarán como un comentario neutral.

In [ ]:
# Se eliminan las columnas 'reviews_funny' y 'reviews_last_edited'
df_reviews = df_reviews.drop(columns=['reviews_funny', 'reviews_last_edited'])
df_reviews.columns

Index(['user_id', 'reviews_posted', 'reviews_item_id', 'reviews_helpful',
       'reviews_recommend', 'reviews_review'],
      dtype='object')

## Transformación de la columna 'reviews_posted'

Se necesita que la fecha donde se hizo el posteo de la review este en formato YYYY-MM-DD, pero se encuentra como Posted November 9, 2012.. Por lo tanto, es necesario procesar la fecha y extraer los elementos relevantes. Se utilizará expresiones regulares para buscar y capturar los valores de año, mes y día dentro de la cadena de texto.

In [ ]:
# Visualizar los datos de la columna 'reviews_posted'
unique_posted_values = df_reviews['reviews_posted'].unique()

for value in unique_posted_values:
    print(value)

Posted July 15, 2011.
Posted September 8, 2013.
Posted December 4, 2015.
Posted July 28, 2012.
Posted December 23, 2013.
Posted December 24, 2012.
Posted June 20, 2014.
Posted August 25, 2014.
Posted March 30, 2015.
Posted April 27, 2013.
Posted September 20, 2014.
Posted February 9, 2014.
Posted August 23.
Posted May 16.
Posted May 19, 2015.
Posted August 21, 2015.
Posted May 29, 2014.
Posted December 26, 2014.
Posted June 16.
Posted August 24, 2015.
Posted September 11, 2015.
Posted May 27, 2012.
Posted July 28.
Posted May 7.
Posted December 2, 2015.
Posted November 14, 2014.
Posted December 15, 2013.
Posted September 13, 2015.
Posted June 25, 2013.
Posted February 22, 2014.
Posted August 3, 2015.
Posted July 9, 2014.
Posted July 21, 2015.
Posted January 15.
Posted August 21, 2014.
Posted March 7, 2015.
Posted February 19, 2014.
Posted December 24, 2015.
Posted April 5, 2014.
Posted August 1, 2014.
Posted September 14.
Posted April 11, 2015.
Posted December 29, 2013.
Posted January 2

In [ ]:
# Asegúrate de que 'reviews_posted' sea de tipo datetime
df_reviews['reviews_posted'] = pd.to_datetime(df_reviews['reviews_posted'].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')

# Crea la columna 'year' a partir de 'reviews_posted'
df_reviews['year'] = df_reviews['reviews_posted'].dt.year.astype('Int64')

# Ordena el DataFrame por 'item_id' y 'year' para asegurar que la interpolación se haga correctamente
ddf_reviews = df_reviews.sort_values(['reviews_item_id', 'year'])

In [ ]:
ddf_reviews

,user_id,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,year
27593,76561198040188061,2011-05-18,10,0 of 2 people (0%) found this review helpful,True,this game is the 1# online action game is awes...,2011
110722,maddoxx789,2012-07-24,10,No ratings yet,True,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZÁMIT !!...,2012
116089,mixadance,2012-09-27,10,2 of 3 people (67%) found this review helpful,True,:D,2012
18059,666_pl_Satan_pl_666,2013-11-24,10,2 of 2 people (100%) found this review helpful,True,"Ogólnie fajna gra, trudno nie polecić. Jedyny ...",2013
69411,epic_doom,2013-07-05,10,1 of 1 people (100%) found this review helpful,True,The OG to CS:GO.,2013
...,...,...,...,...,...,...,...
73237,76561198073638107,NaT,99900,1 of 4 people (25%) found this review helpful,False,Elite orbs are more rarer than good presidenti...,<NA>
78864,ironhorse612,NaT,99900,0 of 2 people (0%) found this review helpful,True,1. Go On Steam.2. Download Spiral Knights.3. P...,<NA>
110741,GOLDENFREDRICKSTAR,NaT,99900,1 of 1 people (100%) found this review helpful,True,Great game.,<NA>
4793,Gatsukama,2011-12-28,99910,No ratings yet,True,"Looks like a cute childrens game, but has more...",2011


In [ ]:
# Rellenar valores nulos en 'year' mediante interpolación lineal por grupo (reviews_item_id	)
ddf_reviews['year'] = ddf_reviews.groupby('reviews_item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)

# Si aún hay valores nulos después de la interpolación, se llenan con la mediana.
ddf_reviews['year'] = ddf_reviews['year'].fillna(ddf_reviews['year'].median())

Identificamos que la columna 'posted' contenía algunos valores estaban ausentes en cuanto al año. Dado que la temporalidad es esencial para nuestro análisis, decidimos abordar este problema de manera estratégica una vez creada la columna 'year'.

- Selección de Técnica de Imputación:
Optamos por utilizar la técnica de interpolación para llenar los valores faltantes en 'year'. La interpolación es útil cuando existe una relación secuencial o temporal en los datos.

- Agrupación por Juego ('item_id'):
Dado que cada review está asociado a un juego único, agrupamos los datos por 'item_id' para considerar la relación temporal dentro de cada juego.

- Aplicación de Interpolación Lineal por Grupo:
Aplicamos la interpolación lineal a cada grupo de 'item_id', lo que permitió estimar los años faltantes basándonos en los años conocidos del mismo juego. Se consideró la opción de aplicar la interpolación lineal global, pero optamos por la interpolación por grupo para capturar posibles variaciones en la temporalidad entre diferentes juegos).


- Manejo de Valores Nulos Restantes:
Después de la interpolación, si aún había valores nulos, llenamos esos espacios con la mediana de la columna 'year'.
Este proceso asegura que nuestro conjunto de datos mantenga la coherencia temporal necesaria para análisis posteriores, y la elección de la interpolación por grupo se alinea con la naturaleza de los datos, donde la temporalidad puede variar entre diferentes juegos. NOTA: Para una 2da versión los registos nulos restantes se podrían llenar con inicialmente con el año de publicación del juego 'release_date' en el archivo steam_games.json y por último con la mediana.

Ahora, evaluaremos la columna 'review' para obtener estadísticas sobre la cantidad de reseñas por lenguaje. Este análisis será crucial para determinar si el proceso de análisis de sentimientos se realizará en todo el conjunto de datos o si se aplicará un filtro específico por idioma.  

In [ ]:
# Análisis de texto para determinar el idioma de las reseñas

def detectar_idioma(texto):
    try:
        return detect(texto)
    except:
        return None

# Aplicar la función para detectar idioma y crear una nueva columna 'language'
ddf_reviews['language'] = ddf_reviews['reviews_review'].apply(detectar_idioma)

# Calcular el conteo y porcentaje de cada idioma
conteo_por_idioma = ddf_reviews['language'].value_counts()
porcentaje_por_idioma = ddf_reviews['language'].value_counts(normalize=True) * 100

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_idiomas = pd.DataFrame({
    'Conteo': conteo_por_idioma,
    'Porcentaje': porcentaje_por_idioma.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_idiomas = resumen_idiomas.sort_values(by='Conteo', ascending=False)
resumen_idiomas.head()

,Conteo,Porcentaje
en,25982,79.52%
pt,1134,3.47%
es,664,2.03%
de,615,1.88%
so,465,1.42%


In [ ]:
#esto es para tener la información en el EDA

# Mapeo de códigos de idioma a nombres completos (Top 5)
mapeo_idiomas = {
    'en': 'English',
    'pt': 'Portugués',
    'es': 'Español',
    'de': 'German',
    'so': 'Somali',
    # Agrega más mapeos según sea necesario
}

# Aplicar el mapeo al DataFrame
ddf_reviews['idioma_completo'] = ddf_reviews['language'].map(mapeo_idiomas)

# Crear un DataFrame con el resumen de idiomas
resumen_idiomas = ddf_reviews['idioma_completo'].value_counts().reset_index()
resumen_idiomas.columns = ['Idioma', 'Conteo']
resumen_idiomas['Conteo'] = resumen_idiomas['Conteo'].round(2)

# Calcular el porcentaje
resumen_idiomas['Porcentaje'] = (resumen_idiomas['Conteo'] / len(ddf_reviews)) * 100
resumen_idiomas['Porcentaje'] = resumen_idiomas['Porcentaje'].round(2)


In [ ]:
# Especifica el nombre del archivo CSV y la ruta donde deseas guardarlo
nombre_archivo_csv = 'ddf_reviews_resumen_idiomas.csv'
ruta_guardado = '/content/drive/MyDrive/'

# Combina la ruta y el nombre del archivo
ruta_completa = ruta_guardado + nombre_archivo_csv

# Exporta el DataFrame a un archivo CSV
resumen_idiomas.to_csv(ruta_completa, index=False)

# Imprime un mensaje de confirmación
print(f"DataFrame exportado exitosamente a: {ruta_completa}")

DataFrame exportado exitosamente a: /content/drive/MyDrive/ddf_reviews_resumen_idiomas.csv


In [ ]:
# Filtrar los registros donde language no es igual a 'en'
ddf_reviews = ddf_reviews[ddf_reviews['idioma_completo'] == 'English']

# Borrar la columna 'idioma_completo'
ddf_reviews = ddf_reviews.drop('idioma_completo', axis=1)

Con los resultados obtenidos consideré realizar el análisis de sentimiento solo en los registros en inglés. Razones que justifican esta decisión (en el EDA se profundiza un poco más):

- Mayor Representatividad.
- Precisión del Modelo.
- Eficiencia Computacional.

**Feature Engineering**

La columna 'review' también será parte de nuestro estudio ya que incluye reseñas de juegos hechos por distintos usuarios.
Partiendo de 'review' se va crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: Debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. De no ser posible este análisis por estar ausente la reseña escrita, tomará el valor de 1.
Esta nueva columna reemplaza la de 'review' para facilitar el trabajo de los modelos de machine learning y el análisis de datos.

In [ ]:
#Se convierten todas las letras a minúsculas para asegurar que todas las palabras sean tratadas de la misma manera.
ddf_reviews.loc[:, 'reviews_review'] = ddf_reviews['reviews_review'].str.lower()

In [ ]:
#Cambiar el nombre de la columna a review
ddf_reviews.rename(columns={'reviews_review': 'review'}, inplace=True)

In [ ]:
#Eliminación de caracteres especiales
ddf_reviews['review'] = ddf_reviews['review'].replace('[^A-Za-z0-9\s]+', '', regex=True)

In [ ]:
#Elimina caracteres de puntuación que no aportan al análisis de sentimiento.
ddf_reviews.loc[:, 'review'] = ddf_reviews['review'].str.replace('[^\w\s]', '', regex=True)

In [ ]:
# Análisis de sentimiento para categorizar las reseñas columna 'review'

def analyze_sentiments(df):
    # Instanciar el analizador de sentimientos
    sia = SentimentIntensityAnalyzer()

    # Aplicar el análisis de sentimientos y asignar valores numéricos
    df['compound_score'] = df['review'].apply(lambda review: sia.polarity_scores(review)['compound'])
    df['sentiment_analysis'] = df['compound_score'].apply(lambda score: 0 if score < 0 else (1 if score == 0 else 2))

    # Conteo de reviews por score
    score_counts = df['sentiment_analysis'].value_counts()

    # Conteo de reviews en blanco
    blank_reviews_count = df['review'].isnull().sum()

    # Total de reviews
    total_reviews = len(df)

    # Calcular porcentajes
    score_percentages = (score_counts / total_reviews * 100).round(2)
    blank_reviews_percentage = (blank_reviews_count / total_reviews * 100).round(2)

    return df, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage

In [ ]:
# Llamar a la función analyze_sentiments
ddf_reviews, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage = analyze_sentiments(ddf_reviews)

In [ ]:
    # Se eliminan las columnas 'review' y 'compound_score', no necesitaremos estos datos
    # df_reviews.drop(['review','compound_score'], axis=1, inplace=True)

In [ ]:
# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_sentimientos = pd.DataFrame({
    'Conteo': score_counts,
    'Porcentaje': score_percentages.round(2).astype(str) + '%'
})

In [ ]:
# Ordenar el DataFrame por el conteo de mayor a menor
resumen_sentimientos = resumen_sentimientos.sort_values(by='Conteo', ascending=False)

In [ ]:
# Imprimir los resultados
print("\nResumen de análisis de sentimientos:")
print(resumen_sentimientos)
print("\nConteo de reviews en blanco: ", blank_reviews_count, " Porcentaje: ", blank_reviews_percentage.round(2).astype(str) + '%')


Resumen de análisis de sentimientos:
   Conteo Porcentaje
2   18127     69.77%
0    5053     19.45%
1    2802     10.78%

Conteo de reviews en blanco:  0  Porcentaje:  0.0%


In [ ]:
# Borrar la columna 'language'
ddf_reviews = ddf_reviews.drop('language', axis=1)
ddf_reviews

KeyError: "['language'] not found in axis"

In [ ]:
#Cambiar el nombre de la columna a item_id
ddf_reviews.rename(columns={'reviews_item_id': 'item_id'}, inplace=True)

In [ ]:
#Cambiar el nombre de la columna a recommend
ddf_reviews.rename(columns={'reviews_recommend': 'recommend'}, inplace=True)

In [ ]:
#Cambiar el nombre de la columna a posted
ddf_reviews.rename(columns={'reviews_posted': 'posted'}, inplace=True)

In [ ]:
ddf_reviews

,user_id,posted,item_id,reviews_helpful,recommend,review,year,compound_score,sentiment_analysis
27593,76561198040188061,2011-05-18,10,0 of 2 people (0%) found this review helpful,True,this game is the 1 online action game is aweso...,2011,0.7906,2
69411,epic_doom,2013-07-05,10,1 of 1 people (100%) found this review helpful,True,the og to csgo,2013,0.0000,1
3034,mayshowganmore,2014-01-22,10,0 of 2 people (0%) found this review helpful,True,the best fps game,2014,0.6369,2
5286,Bigzy,2014-09-01,10,1 of 2 people (50%) found this review helpful,True,first online shooter i played,2014,0.3400,2
11159,NicolasNic,2014-06-21,10,No ratings yet,True,20122014 awsome classic game changing shooter...,2014,0.0516,2
...,...,...,...,...,...,...,...,...,...
73237,76561198073638107,NaT,99900,1 of 4 people (25%) found this review helpful,False,elite orbs are more rarer than good presidenti...,2015,0.4877,2
78864,ironhorse612,NaT,99900,0 of 2 people (0%) found this review helpful,True,1 go on steam2 download spiral knights3 play t...,2015,0.3400,2
110741,GOLDENFREDRICKSTAR,NaT,99900,1 of 1 people (100%) found this review helpful,True,great game,2015,0.6249,2
4793,Gatsukama,2011-12-28,99910,No ratings yet,True,looks like a cute childrens game but has more ...,2011,0.7311,2


In [ ]:
ddf_reviews = ddf_reviews.dropna(subset=['posted'])

valores_nulos = ddf_reviews.isnull().sum()

# Imprime el resultado
print("Valores nulos por columna:")
print(valores_nulos)

Valores nulos por columna:
user_id               0
posted                0
item_id               0
reviews_helpful       0
recommend             0
review                0
year                  0
compound_score        0
sentiment_analysis    0
dtype: int64


**Exportar JSON a CSV**

In [ ]:
# Especifica el nombre del archivo CSV y la ruta donde deseas guardarlo
nombre_archivo_csv = 'data_reviews_cleaned.csv'
ruta_guardado = '/content/drive/MyDrive/'

# Combina la ruta y el nombre del archivo
ruta_completa = ruta_guardado + nombre_archivo_csv

# Exporta el DataFrame a un archivo CSV
ddf_reviews.to_csv(ruta_completa, index=False)

# Imprime un mensaje de confirmación
print(f"DataFrame exportado exitosamente a: {ruta_completa}")

DataFrame exportado exitosamente a: /content/drive/MyDrive/data_reviews_cleaned.csv


**Cargar y transformar archivo JSON:** *output_steam_games*

In [ ]:
#Carga y muestra el archivo
data_games = pd.read_json('/content/drive/MyDrive/JSON/output_steam_games.json', lines=True)
data_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [ ]:
#Muestra los datos no nulos del dataFrame
data_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [ ]:
# Muestra el porcentajes de datos nulos por columnas
porcentajes_nulos = (data_games.isnull().mean() * 100).round(2)
porcentajes_nulos

publisher       80.00
genres          76.05
app_name        73.32
title           75.02
url             73.32
release_date    75.04
tags            73.46
reviews_url     73.32
specs           73.88
price           74.46
early_access    73.32
id              73.32
developer       76.06
dtype: float64

In [ ]:
# Verificar si hay filas vacias en el dataframe
filas_vacias_games = data_games.isnull().all(axis=1)
filas_vacias_games

0          True
1          True
2          True
3          True
4          True
          ...  
120440    False
120441    False
120442    False
120443    False
120444    False
Length: 120445, dtype: bool

In [ ]:
# Elimina las filas que tiene todos los valores nulos
data_games_final=data_games.dropna(how='all')
data_games_final

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [ ]:
data_games_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.4+ MB


In [ ]:
# Muestra el porcentajes de datos nulos por columnas
porcentajes_nulos = (data_games_final.isnull().mean() * 100).round(2)
porcentajes_nulos

publisher       25.06
genres          10.22
app_name         0.01
title            6.38
url              0.00
release_date     6.43
tags             0.51
reviews_url      0.01
specs            2.08
price            4.29
early_access     0.00
id               0.01
developer       10.27
dtype: float64

---

**Exportar JSON a CSV**

---

In [ ]:
# Especifica el nombre del archivo CSV y la ruta donde deseas guardarlo
nombre_archivo_csv = 'data_games_final.csv'
ruta_guardado = '/content/drive/MyDrive/Dataset JSON'

# Combina la ruta y el nombre del archivo
ruta_completa = ruta_guardado + nombre_archivo_csv

# Exporta el DataFrame a un archivo CSV
data_games_final.to_csv(ruta_completa, index=False)

# Imprime un mensaje de confirmación
print(f"DataFrame exportado exitosamente a: {ruta_completa}")

DataFrame exportado exitosamente a: /content/drive/MyDrive/Dataset JSONdata_games_final.csv


---

**Cargar y transformar archivo JSON:** *australian_users_items*

---

In [ ]:
#Carga y muestra el archivo
df_items = load_json_lines('/content/drive/MyDrive/JSON/australian_users_items.json')
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [ ]:
#Muestra los datos no nulos del dataFrame
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [ ]:
# Muestra el porcentajes de datos nulos por columnas
porcentajes_nulos = (df_items.isnull().mean() * 100).round(2)
porcentajes_nulos

user_id        0.0
items_count    0.0
steam_id       0.0
user_url       0.0
items          0.0
dtype: float64

In [ ]:
# Explora las columnas del DataFrame
df_items.head   # Se observa que la columna items es una columna anidada

<bound method NDFrame.head of                        user_id  items_count           steam_id  \
0            76561197970982479          277  76561197970982479   
1                      js41637          888  76561198035864385   
2                    evcentric          137  76561198007712555   
3                   Riot-Punch          328  76561197963445855   
4                        doctr          541  76561198002099482   
...                        ...          ...                ...   
88305        76561198323066619           22  76561198323066619   
88306        76561198326700687          177  76561198326700687   
88307  XxLaughingJackClown77xX            0  76561198328759259   
88308        76561198329548331            7  76561198329548331   
88309        edward_tremethick            0  76561198331598578   

                                                user_url  \
0      http://steamcommunity.com/profiles/76561197970...   
1                   http://steamcommunity.com/id/js41637 

In [ ]:
# Inicializa una lista para almacenar los DataFrames normalizados
lista_dataframes = []

# Itera sobre las filas de la columna 'items'
for index, fila in df_items['items'].iteritems():
    # Normaliza la fila actual y agrega el DataFrame resultante a la lista
    df_normalizado = pd.json_normalize(fila)
    lista_dataframes.append(df_normalizado)

In [ ]:
# Concatena todos los DataFrames en uno solo
data_items_col_items = pd.concat(lista_dataframes, ignore_index=True)
data_items_col_items

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6,0
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,7,0
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0
...,...,...,...,...
5153204,346330,BrainBread 2,0,0
5153205,373330,All Is Dust,0,0
5153206,388490,One Way To Die: Steam Edition,3,3
5153207,521570,You Have 10 Seconds 2,4,4


In [ ]:
# Elimina la columna items anidada del DataFrame
df_items_sin_col_items_anidada = df_items.drop(columns=['items'])
df_items_sin_col_items_anidada

,user_id,items_count,steam_id,user_url
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr
...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [ ]:
# Concatena el DataFrame original con la versión aplanada de la columna items
data_items = pd.concat([df_items_sin_col_items_anidada, data_items_col_items], axis=1)
data_items

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,js41637,888.0,76561198035864385,http://steamcommunity.com/id/js41637,20,Team Fortress Classic,0,0
2,evcentric,137.0,76561198007712555,http://steamcommunity.com/id/evcentric,30,Day of Defeat,7,0
3,Riot-Punch,328.0,76561197963445855,http://steamcommunity.com/id/Riot-Punch,40,Deathmatch Classic,0,0
4,doctr,541.0,76561198002099482,http://steamcommunity.com/id/doctr,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...,...,...,...
5153204,NaN,NaN,NaN,NaN,346330,BrainBread 2,0,0
5153205,NaN,NaN,NaN,NaN,373330,All Is Dust,0,0
5153206,NaN,NaN,NaN,NaN,388490,One Way To Die: Steam Edition,3,3
5153207,NaN,NaN,NaN,NaN,521570,You Have 10 Seconds 2,4,4


In [ ]:
data_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       float64
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  int64  
 7   playtime_2weeks   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 314.5+ MB


In [ ]:
# Muestra el porcentajes de datos nulos por columnas
porcentajes_nulos = (data_items.isnull().mean() * 100).round(2)
porcentajes_nulos

user_id             98.29
items_count         98.29
steam_id            98.29
user_url            98.29
item_id              0.00
item_name            0.00
playtime_forever     0.00
playtime_2weeks      0.00
dtype: float64

In [ ]:
# Elimina todas las columnas con un total de datos nulos mayor a 85%
columnas_a_eliminar = porcentajes_nulos[porcentajes_nulos > 85].index
data_items_final = data_items.drop(columns=columnas_a_eliminar)
data_items_final

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6,0
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,7,0
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0
...,...,...,...,...
5153204,346330,BrainBread 2,0,0
5153205,373330,All Is Dust,0,0
5153206,388490,One Way To Die: Steam Edition,3,3
5153207,521570,You Have 10 Seconds 2,4,4


In [ ]:
# Muestra el porcentajes de datos nulos por columnas
porcentajes_nulos = (data_items_final.isnull().mean() * 100).round(2)
porcentajes_nulos

item_id             0.0
item_name           0.0
playtime_forever    0.0
playtime_2weeks     0.0
dtype: float64

---

**Exportar JSON a CSV**

---

In [ ]:
# Especifica el nombre del archivo CSV y la ruta donde deseas guardarlo
nombre_archivo_csv = 'data_items_final.csv'
ruta_guardado = '/content/drive/MyDrive/Dataset JSON'

# Combina la ruta y el nombre del archivo
ruta_completa = ruta_guardado + nombre_archivo_csv

# Exporta el DataFrame a un archivo CSV
data_items_final.to_csv(ruta_completa, index=False)

# Imprime un mensaje de confirmación
print(f"DataFrame exportado exitosamente a: {ruta_completa}")

DataFrame exportado exitosamente a: /content/drive/MyDrive/Dataset JSONdata_items_final.csv
